In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
import time

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
num_classes = 50
batch_size = 64  # Increased batch size
num_epochs = 10
learning_rate = 0.001
num_workers = 8
train_split = 0.8  # 80% of data for training

# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load data
full_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/JPEGImages', transform=transform)

# Split data into train and test sets
train_size = int(train_split * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)



In [4]:
# Model
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Replace the last layer
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # Learning rate scheduler

# Initialize gradient scaler for mixed precision
scaler = amp.GradScaler()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 169MB/s]


In [8]:
# Train the model
def train_model():
    model.train()
    start_time = time.time()
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            with amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if (i+1) % 20 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

        scheduler.step()  # Update learning rate
    print(f'Training time: {time.time() - start_time:.2f} seconds')





In [9]:
# Evaluate the model
def evaluate_model():
    model.eval()
    with torch.no_grad(), amp.autocast():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy of the model on test images: {100 * correct / total:.2f}%')



In [10]:
if __name__ == "__main__":
    train_model()
    evaluate_model()

Epoch [1/10], Step [20/129], Loss: 0.2453
Epoch [1/10], Step [40/129], Loss: 0.1680
Epoch [1/10], Step [60/129], Loss: 0.2478
Epoch [1/10], Step [80/129], Loss: 0.2665
Epoch [1/10], Step [100/129], Loss: 0.2005
Epoch [1/10], Step [120/129], Loss: 0.1839
Epoch [2/10], Step [20/129], Loss: 0.1938
Epoch [2/10], Step [40/129], Loss: 0.1777
Epoch [2/10], Step [60/129], Loss: 0.0895
Epoch [2/10], Step [80/129], Loss: 0.1658
Epoch [2/10], Step [100/129], Loss: 0.1734
Epoch [2/10], Step [120/129], Loss: 0.2559
Epoch [3/10], Step [20/129], Loss: 0.1568
Epoch [3/10], Step [40/129], Loss: 0.1153
Epoch [3/10], Step [60/129], Loss: 0.2181
Epoch [3/10], Step [80/129], Loss: 0.0975
Epoch [3/10], Step [100/129], Loss: 0.1340
Epoch [3/10], Step [120/129], Loss: 0.1119
Epoch [4/10], Step [20/129], Loss: 0.1199
Epoch [4/10], Step [40/129], Loss: 0.1242
Epoch [4/10], Step [60/129], Loss: 0.2128
Epoch [4/10], Step [80/129], Loss: 0.1917
Epoch [4/10], Step [100/129], Loss: 0.2520
Epoch [4/10], Step [120/129